# Preliminary baseline results

In [1]:
import os
import pickle
from pathlib import Path
from functools import partial

import pandas as pd
from bokeh.io import show, output_notebook
output_notebook()

from liftoff.liftoff_results import collect_results

from lineplot import simple_plot, lineplot
from panda_utils import filter_by, make_df

collect_results = partial(collect_results, results_dir='../results', names=['testing.pkl'])

Loading BokehJS ...

### Fetch the results paths

In [2]:
def make_key(path):
    algo = path.parts[2].split('_')[-1]
    game_name = path.parts[3][7:]
    trial = path.parts[4]
    return '-'.join([game_name, algo, trial])

In [3]:
experiments = ("two_way_dqn", "two_way_ddqn", "two_way_per")

exp_results = [collect_results(experiment_name=exp_name) for exp_name in experiments]
exp_results = [trial for exp in exp_results for trial in exp]

file_paths = [Path(f"{res[0]}/testing.pkl") for res in exp_results if res[1]]

### Load data

In [4]:
results_dict = {make_key(path): pickle.load(open(path, 'rb')) for path in file_paths}
for k in results_dict.keys():
    print(k)
    
data = make_df(results_dict)

smoothed = (data.assign(smooth_rw_per_ep=data.groupby(['game','algo','trial'], as_index=False)[['rw_per_ep']]
                           .rolling(5, min_periods=1).mean().fillna(0)
                           .reset_index(0, drop=True)))

smoothed.sample(10)

seaquest-dqn-2
seaquest-dqn-0
seaquest-dqn-1
road_runner-dqn-2
road_runner-dqn-0
road_runner-dqn-1
seaquest-ddqn-2
seaquest-ddqn-0
seaquest-ddqn-1
road_runner-ddqn-2
road_runner-ddqn-0
road_runner-ddqn-1
seaquest-per-2
seaquest-per-0
seaquest-per-1
road_runner-per-2
road_runner-per-0
road_runner-per-1


,step_idx,time_idx,ep_cnt,rw_per_ep,step_per_ep,rw_per_step,max_q,test_fps,game,algo,trial,smooth_rw_per_ep
224,5500000,25427.663496,100,1030.1,1373.41,0.750031,34.697021,58.793337,seaquest,ddqn,0,1484.28
574,7500000,33022.250556,100,38890.0,2779.68,13.990819,26.142845,113.429669,road_runner,per,1,29855.20
324,18500000,92327.400691,100,17365.0,4239.85,4.095664,15.810931,137.023367,road_runner,ddqn,2,14913.20
110,12000000,53336.788514,100,10.0,597.70,0.016731,2412.054199,26.874764,road_runner,dqn,2,19.60
389,7500000,34252.735864,100,3380.3,2849.83,1.186141,29.596409,114.809863,seaquest,per,2,2733.18
415,2000000,8033.230750,100,1049.0,1923.69,0.545306,4.994894,90.412810,seaquest,per,0,626.75
157,17000000,73840.299092,100,467.0,509.56,0.916477,225.586334,22.908606,road_runner,dqn,0,208.80
123,18500000,81400.511815,100,895.0,941.56,0.950550,5198.514160,41.012649,road_runner,dqn,2,185.60
317,15000000,73052.425774,100,4932.0,328.70,15.004563,14.992627,15.032727,road_runner,ddqn,2,12944.60
94,4000000,17267.584003,100,0.0,320.08,0.000000,58.682362,14.838783,road_runner,dqn,2,137.80


## Seaquest

### Smoothed Data

In [5]:
show(lineplot(filter_by(smoothed, 'seaquest', ['per', 'ddqn', 'dqn']),
              'step_idx', 'smooth_rw_per_ep', 'algo', 'trial', aggregate=True, title="Smoothed Seaquest PER"))

In [6]:
show(lineplot(filter_by(smoothed, 'seaquest'),
              'step_idx', 'smooth_rw_per_ep', 'algo', aggregate=True, title="Smoothed Seaquest PER"))

### Raw data

In [7]:
show(lineplot(filter_by(data, 'seaquest', ['per', 'ddqn', 'dqn']),
              'step_idx', 'rw_per_ep', 'algo', 'trial', aggregate=True, title="Seaquest DDQN vs PER"))

## RoadRunner

### Smoothed data

In [8]:
show(lineplot(filter_by(smoothed, 'road_runner'),
              'step_idx', 'smooth_rw_per_ep', 'algo', 'trial', aggregate=True, title="RoadRunner - DQN vs DDQN vs PER"))

In [9]:
show(lineplot(filter_by(smoothed, 'road_runner'),
              'step_idx', 'smooth_rw_per_ep', 'algo', aggregate=True, title="RoadRunner - DQN vs DDQN vs PER"))

### Raw data

In [10]:
show(lineplot(filter_by(data, 'road_runner'),
              'step_idx', 'rw_per_ep', 'algo', 'trial', aggregate=True, title="RoadRunner - DQN vs DDQN vs PER"))